In [1]:
import numpy as np
import tensorflow as tf
from random import sample
from captcha.image import ImageCaptcha
from reader import CaptchaReader

In [2]:
width = 100
height = 60
charset = '0123456789'
def getCode():
    return ''.join(map(lambda x: charset[x], np.random.randint(0, 10, 4)))
captcha = ImageCaptcha(width=width, height=height)

In [3]:
def getData(n = 10):    
    codeList = [getCode() for _ in range(n)]
    imageList = map(lambda code: captcha.generate_image(code), codeList)
    return imageList, codeList

In [4]:
reader = CaptchaReader(width, height, 4, charset)

In [5]:
session=tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('captcha-1000.meta')
saver.restore(session,tf.train.latest_checkpoint('./'))

INFO:tensorflow:Restoring parameters from ./captcha-1000


In [6]:
for i, _ in enumerate(range(10000), 1):
    imageList, codeList = getData(100)
    batch_data = np.array(map(reader.imageToVertor, imageList))
    batch_labels = np.array(map(reader.codeToVertor, codeList))
    reader.step += 1
    _, loss = session.run(
        [reader.optimizer, reader.loss],
        feed_dict={
            reader.x: batch_data,
            reader.y: batch_labels,
            reader.keep_prob: 0.75
        }   
    ) 
    print u'[%d]: loss: %f' % (i, loss)

TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("Placeholder:0", shape=(?, 6000), dtype=float32) is not an element of this graph.